<a href="https://colab.research.google.com/github/mtzig/badwriting_OCR/blob/main/Reptile_MAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!git clone https://github.com/mtzig/badwriting_OCR.git
%cd /content/badwriting_OCR/

Cloning into 'badwriting_OCR'...
remote: Enumerating objects: 376, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 376 (delta 79), reused 127 (delta 47), pack-reused 192
Receiving objects: 100% (376/376), 8.40 MiB | 4.71 MiB/s, done.
Resolving deltas: 100% (124/124), done.
/content/badwriting_OCR


In [3]:
!pip install -q transformers
!pip install -q datasets jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.1 MB/s eta 0:00:00


In [4]:
# from utils import model_utils
from transformers import AdamW, VisionEncoderDecoderModel,TrOCRProcessor
from tqdm.notebook import tqdm
import torch
from PIL import Image
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [5]:
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-small-handwritten")

outer_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-small-handwritten")
# model.to(device)

outer_optimizer = AdamW(outer_model.parameters(), lr=5e-5)


preprocessor_config.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/246M [00:00<?, ?B/s]

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
# set special tokens used for creating the decoder_input_ids from the labels
outer_model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
outer_model.config.pad_token_id = processor.tokenizer.pad_token_id
# make sure vocab size is set correctly
outer_model.config.vocab_size = outer_model.config.decoder.vocab_size

In [7]:
test_img = Image.open('./data/test_0.png').convert("RGB")
pixel_values = processor(test_img, return_tensors="pt").pixel_values.to(device)

labels = processor.tokenizer('Last time A symmetric operator', padding="max_length",
                                  max_length=128).input_ids
labels = torch.tensor([label if label != processor.tokenizer.pad_token_id else -100 for label in labels], device=device).unsqueeze(0)
# labels = torch.tensor([label if label != processor.tokenizer.pad_token_id else -100 for label in labels]).unsqueeze(0)



In [8]:
pixel_values.shape

torch.Size([1, 3, 384, 384])

In [9]:
inner_model = deepcopy(outer_model)
inner_model.to(device)
inner_optimizer = AdamW(inner_model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [10]:
inner_model.train()
loss = inner_model(pixel_values=pixel_values, labels=labels).loss
loss.backward()
inner_optimizer.step()
inner_optimizer.zero_grad()

In [12]:
grads = [o-i.to('cpu') for (o,i) in zip(outer_model.parameters(), inner_model.parameters())]
# need to sum these over all tasks and normalize by dividing over number of tasks

In [13]:
for i, params in enumerate(outer_model.parameters()):
    params.grad = grads[i]

In [14]:
outer_optimizer.step()
outer_optimizer.zero_grad()

See [this repo](https://github.com/nabarunbaruaAIML/Meta_Learning/blob/master/Meta_Learning.py) for reference on Reptile MAML implementation.